# Conversational RAG
A conversational LLM app using LangChain & Chroma. Built from these tutorials: 
- [LangChain RAG](https://python.langchain.com/v0.2/docs/tutorials/rag/)
- [Conversational RAG](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/)

This exercise builds on the `basic-rag.ipynb` exercise and adds in:
- Integration of chat history into subsequent queries, programmatically updated in function calls
- Chain tracing with LangSmith

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4
%pip install -qU langchain-openai
%pip install python-dotenv
%pip install langchain_core

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# Enable tracing with LangSmith
# LANGCHAIN_API_KEY environment variable is set in .env
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = "conversational-rag"

# Set the USER_AGENT environment variable
os.environ['USER_AGENT'] = 'conversational-rag-agent'

## 1. Load, chunk and index source documents to create a retriever.

In [3]:
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(splits, OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

## 2. Add the retriever into a question-answering chain

In [4]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# response = rag_chain.invoke({"input": "How are tools used in LLMs?"})
# e.g. "What is Task Decomposition?"
# e.g. "How are tools used in LLMs?"
# e.g. "What is Chain of Thought reasoning?"

# response["answer"]



## 3. Add a history-aware retriever chain
This chain prepends a rephrasing of the input query to the original retriever, so that the retrieval incorporates the context of the conversation.

In [5]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

## 4. Assemble full conversational chain

In [6]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

## 5. Invoke the chain & extend history

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

while True:
    question = input("> ")
    if question == "q":
        break
    ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.extend(
        [
            HumanMessage(content=question),
            AIMessage(content=ai_msg["answer"]),
        ]
    )
    print(ai_msg["answer"])

# e.g. "What is Task Decomposition?"
# e.g. "How are tools used in LLMs?"
# e.g. "What is Chain of Thought reasoning?"

In [ ]:
# View chat history
import json

print(json.dumps([message.dict() for message in chat_history], indent=4))